## 1. Setup & Imports

In [ ]:
# Install required packages (uncomment if running in Colab)
# !pip install xgboost lightgbm scikit-learn pandas numpy yfinance joblib -q

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import joblib
import json
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful!")

## 2. Configuration

In [ ]:
# Training configuration
CONFIG = {
    'training_universe_size': 500,  # Number of stocks to train on
    'history_days': 730,            # 2 years of historical data
    'forward_prediction_days': 30,  # Predict 30-day returns
    'test_size': 0.2,               # 20% for testing
    'random_state': 42,
    'output_dir': 'ml_models',      # Where to save models
}

# Model weights for ensemble
MODEL_WEIGHTS = {
    'xgboost': 0.35,
    'random_forest': 0.25,
    'lightgbm': 0.40,
}

print(f"Training on {CONFIG['training_universe_size']} stocks")
print(f"Using {CONFIG['history_days']} days of historical data")
print(f"Predicting {CONFIG['forward_prediction_days']}-day returns")

## 3. Load Stock Universe

In [ ]:
# Load stock list from data directory or use default universe
try:
    # Try to load from CSV
    stocks_df = pd.read_csv('../data/eodhd_us_tickers.csv')
    # Filter for liquid stocks with market cap > $1B
    universe = stocks_df['Symbol'].head(CONFIG['training_universe_size']).tolist()
    print(f"✅ Loaded {len(universe)} stocks from CSV")
except:
    # Fallback: Use S&P 500 stocks
    print("⚠️ CSV not found, downloading S&P 500 list...")
    sp500_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    universe = sp500_table['Symbol'].head(CONFIG['training_universe_size']).tolist()
    print(f"✅ Loaded {len(universe)} stocks from S&P 500")

print(f"\nFirst 10 stocks: {universe[:10]}")

## 4. Feature Engineering Functions

In [ ]:
def calculate_technical_features(df):
    """
    Calculate technical indicators from OHLCV data
    Returns 19 technical features
    """
    features = {}
    
    # Price-based features
    features['close'] = df['Close'].iloc[-1]
    features['volume'] = df['Volume'].iloc[-1]
    
    # Returns
    features['return_1d'] = df['Close'].pct_change(1).iloc[-1]
    features['return_5d'] = df['Close'].pct_change(5).iloc[-1]
    features['return_20d'] = df['Close'].pct_change(20).iloc[-1]
    features['return_60d'] = df['Close'].pct_change(60).iloc[-1]
    
    # Volatility
    features['volatility_20d'] = df['Close'].pct_change().rolling(20).std().iloc[-1]
    features['volatility_60d'] = df['Close'].pct_change().rolling(60).std().iloc[-1]
    
    # Moving averages
    features['sma_20'] = df['Close'].rolling(20).mean().iloc[-1]
    features['sma_50'] = df['Close'].rolling(50).mean().iloc[-1]
    features['sma_200'] = df['Close'].rolling(200).mean().iloc[-1]
    
    # Moving average crossovers
    features['sma_20_50_ratio'] = features['sma_20'] / features['sma_50'] if features['sma_50'] > 0 else 1
    features['sma_50_200_ratio'] = features['sma_50'] / features['sma_200'] if features['sma_200'] > 0 else 1
    
    # RSI (14-day)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    features['rsi_14'] = 100 - (100 / (1 + rs.iloc[-1]))
    
    # Bollinger Bands
    bb_middle = df['Close'].rolling(20).mean()
    bb_std = df['Close'].rolling(20).std()
    bb_upper = bb_middle + (bb_std * 2)
    bb_lower = bb_middle - (bb_std * 2)
    features['bb_position'] = (df['Close'].iloc[-1] - bb_lower.iloc[-1]) / (bb_upper.iloc[-1] - bb_lower.iloc[-1]) if (bb_upper.iloc[-1] - bb_lower.iloc[-1]) > 0 else 0.5
    
    # Volume features
    features['volume_20d_avg'] = df['Volume'].rolling(20).mean().iloc[-1]
    features['volume_ratio'] = features['volume'] / features['volume_20d_avg'] if features['volume_20d_avg'] > 0 else 1
    
    # Price momentum
    features['momentum_20d'] = df['Close'].iloc[-1] / df['Close'].iloc[-20] - 1 if len(df) >= 20 else 0
    
    return features

def calculate_forward_return(df, days=30):
    """
    Calculate forward return (label for training)
    """
    if len(df) < days:
        return None
    return (df['Close'].iloc[-1] / df['Close'].iloc[-days] - 1)

print("✅ Feature engineering functions defined")

## 5. Download Historical Data & Build Training Dataset

In [ ]:
print("📊 Downloading historical data...\n")

all_features = []
all_labels = []
failed_tickers = []

end_date = datetime.now()
start_date = end_date - timedelta(days=CONFIG['history_days'])

for i, symbol in enumerate(universe):
    try:
        # Download data
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start_date, end=end_date)
        
        if len(df) < 200:  # Need at least 200 days for features
            failed_tickers.append(symbol)
            continue
        
        # Generate samples by sliding window
        for j in range(200, len(df) - CONFIG['forward_prediction_days']):
            window = df.iloc[:j+1].copy()
            future_window = df.iloc[j:j+CONFIG['forward_prediction_days']+1].copy()
            
            # Calculate features
            features = calculate_technical_features(window)
            features['symbol'] = symbol
            
            # Calculate forward return (label)
            label = calculate_forward_return(future_window, CONFIG['forward_prediction_days'])
            
            if label is not None and not np.isnan(label) and all(not np.isnan(v) and not np.isinf(v) for v in features.values() if isinstance(v, float)):
                all_features.append(features)
                all_labels.append(label)
        
        if (i + 1) % 50 == 0:
            print(f"Processed {i+1}/{len(universe)} stocks, Generated {len(all_features)} samples")
    
    except Exception as e:
        failed_tickers.append(symbol)
        continue

print(f"\n✅ Data collection complete!")
print(f"Total samples: {len(all_features)}")
print(f"Failed tickers: {len(failed_tickers)}")
if failed_tickers:
    print(f"Failed: {failed_tickers[:10]}...")

## 6. Prepare Training Data

In [ ]:
# Convert to DataFrame
features_df = pd.DataFrame(all_features)
labels_series = pd.Series(all_labels, name='forward_return')

print(f"Features shape: {features_df.shape}")
print(f"Labels shape: {labels_series.shape}")

# Separate symbol column
symbols = features_df['symbol']
features_df = features_df.drop('symbol', axis=1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    features_df, labels_series, 
    test_size=CONFIG['test_size'], 
    random_state=CONFIG['random_state']
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Data prepared and scaled")

## 7. Train Models

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

models = {}
results = {}

print("🚀 Training models...\n")

In [ ]:
# 1. XGBoost (35% weight)
print("Training XGBoost...")
xgb_model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=CONFIG['random_state'],
    n_jobs=-1
)
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)

results['xgboost'] = {
    'mse': mean_squared_error(y_test, xgb_pred),
    'mae': mean_absolute_error(y_test, xgb_pred),
    'r2': r2_score(y_test, xgb_pred)
}
models['xgboost'] = xgb_model

print(f"✅ XGBoost trained - R²: {results['xgboost']['r2']:.4f}, MAE: {results['xgboost']['mae']:.4f}")

In [ ]:
# 2. Random Forest (25% weight)
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=CONFIG['random_state'],
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)

results['random_forest'] = {
    'mse': mean_squared_error(y_test, rf_pred),
    'mae': mean_absolute_error(y_test, rf_pred),
    'r2': r2_score(y_test, rf_pred)
}
models['random_forest'] = rf_model

print(f"✅ Random Forest trained - R²: {results['random_forest']['r2']:.4f}, MAE: {results['random_forest']['mae']:.4f}")

In [ ]:
# 3. LightGBM (40% weight)
print("\nTraining LightGBM...")
lgbm_model = LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=CONFIG['random_state'],
    n_jobs=-1,
    verbose=-1
)
lgbm_model.fit(X_train_scaled, y_train)
lgbm_pred = lgbm_model.predict(X_test_scaled)

results['lightgbm'] = {
    'mse': mean_squared_error(y_test, lgbm_pred),
    'mae': mean_absolute_error(y_test, lgbm_pred),
    'r2': r2_score(y_test, lgbm_pred)
}
models['lightgbm'] = lgbm_model

print(f"✅ LightGBM trained - R²: {results['lightgbm']['r2']:.4f}, MAE: {results['lightgbm']['mae']:.4f}")

In [ ]:
# 4. Ensemble predictions
print("\nCalculating ensemble predictions...")
ensemble_pred = (
    MODEL_WEIGHTS['xgboost'] * xgb_pred +
    MODEL_WEIGHTS['random_forest'] * rf_pred +
    MODEL_WEIGHTS['lightgbm'] * lgbm_pred
)

results['ensemble'] = {
    'mse': mean_squared_error(y_test, ensemble_pred),
    'mae': mean_absolute_error(y_test, ensemble_pred),
    'r2': r2_score(y_test, ensemble_pred)
}

print(f"✅ Ensemble - R²: {results['ensemble']['r2']:.4f}, MAE: {results['ensemble']['mae']:.4f}")

## 8. Evaluate Results

In [ ]:
print("\n" + "="*60)
print("📊 MODEL PERFORMANCE SUMMARY")
print("="*60)

results_df = pd.DataFrame(results).T
results_df = results_df[['r2', 'mae', 'mse']]
results_df.columns = ['R² Score', 'MAE', 'MSE']
print(results_df.to_string())

print("\n" + "="*60)
print(f"Best Model: {results_df['R² Score'].idxmax()} (R² = {results_df['R² Score'].max():.4f})")
print("="*60)

## 9. Feature Importance

In [ ]:
# Get feature importance from XGBoost
feature_importance = pd.DataFrame({
    'feature': features_df.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

## 10. Save Models

In [ ]:
import os

# Create output directory
output_dir = f"../{CONFIG['output_dir']}"
os.makedirs(output_dir, exist_ok=True)

print(f"💾 Saving models to {output_dir}/...\n")

# Save models
joblib.dump(models['xgboost'], f"{output_dir}/xgboost.pkl")
print("✅ Saved xgboost.pkl")

joblib.dump(models['random_forest'], f"{output_dir}/random_forest.pkl")
print("✅ Saved random_forest.pkl")

joblib.dump(models['lightgbm'], f"{output_dir}/lightgbm.pkl")
print("✅ Saved lightgbm.pkl")

joblib.dump(scaler, f"{output_dir}/feature_engineer.pkl")
print("✅ Saved feature_engineer.pkl (scaler)")

# Save metadata
metadata = {
    'training_date': datetime.now().isoformat(),
    'num_stocks': len(universe) - len(failed_tickers),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'feature_count': len(features_df.columns),
    'features': list(features_df.columns),
    'model_weights': MODEL_WEIGHTS,
    'performance': results,
    'config': CONFIG
}

with open(f"{output_dir}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ Saved metadata.json")

print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)
print(f"\nModels saved to: {output_dir}/")
print("\nFiles created:")
print("  - xgboost.pkl")
print("  - random_forest.pkl")
print("  - lightgbm.pkl")
print("  - feature_engineer.pkl")
print("  - metadata.json")
print("\nReady to deploy! 🚀")

## 11. Download Models (For Colab Users)

In [ ]:
# Uncomment to download models in Google Colab
# from google.colab import files
# import zipfile

# # Create zip file
# with zipfile.ZipFile('ml_models.zip', 'w') as zipf:
#     for file in ['xgboost.pkl', 'random_forest.pkl', 'lightgbm.pkl', 'feature_engineer.pkl', 'metadata.json']:
#         zipf.write(f"{output_dir}/{file}", file)

# # Download
# files.download('ml_models.zip')
# print("✅ Models downloaded as ml_models.zip")